In [1]:
!pip install textattack[tensorflow]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Directory for Colab (optional, use if running Colab)
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
import os
os.chdir('/content/gdrive/MyDrive/NLP_project')

In [4]:
!pwd

/content/gdrive/MyDrive/NLP_project


In [5]:
import pandas as pd
df = pd.read_csv("data/imdb/bert/textfooler/bert-base-uncased-imdb_textfooler.csv",sep=',')

In [6]:
df.head()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,[[Once]] again Mr. Costner has dragged out a [...,[[Whenever]] again Mr. Costner has dragged out...,0.000241,0.815027,0,1,0,469,Successful
1,[[This]] is an example of why the majority of ...,[[These]] is an example of why the majority of...,0.000183,0.554409,0,1,0,670,Successful
2,"First of all I hate those [[moronic]] rappers,...",First of all I hate those [[senseless]] rapper...,0.000289,0.716852,0,1,0,643,Successful
3,Not even the Beatles could write songs everyon...,Not even the Beatles could write songs everyon...,0.000303,0.960098,0,1,0,441,Successful
4,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.000311,0.952665,0,1,0,301,Successful


In [7]:
len(df)

10000

In [8]:
def clean_text(text):
  text = text.replace("[[", "")
  text = text.replace("]]", "")
  return text

In [9]:
df['original_text'] = df['original_text'].map(clean_text)
df['perturbed_text'] = df['perturbed_text'].map(clean_text)
df.head()

,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,Once again Mr. Costner has dragged out a movie...,Whenever again Mr. Costner has dragged out a s...,0.000241,0.815027,0,1,0,469,Successful
1,This is an example of why the majority of acti...,These is an example of why the majority of act...,0.000183,0.554409,0,1,0,670,Successful
2,"First of all I hate those moronic rappers, who...","First of all I hate those senseless rappers, w...",0.000289,0.716852,0,1,0,643,Successful
3,Not even the Beatles could write songs everyon...,Not even the Beatles could write songs everyon...,0.000303,0.960098,0,1,0,441,Successful
4,Brass pictures (movies is not a fitting word f...,Brass pictures (movies is not a fitting word f...,0.000311,0.952665,0,1,0,301,Successful


In [10]:
from transformers import BertModel, BertTokenizer

model_name = 'textattack/bert-base-uncased-imdb'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at textattack/bert-base-uncased-imdb were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
from bert_embeddings import tp_bert
import torch

In [15]:
# Check if you can access to GPU:
torch.cuda.is_available()

True

In [16]:
tp_bert(df['original_text'], model.cuda(), tokenizer, device)

array([[-0.35777315,  0.81997323,  0.42115557, ..., -0.00659452,
         0.02114969, -0.19817111],
       [ 0.13449861,  0.7246046 ,  0.41309497, ...,  0.01864269,
         0.13467726, -0.06292512],
       [-0.12180577,  0.8502505 ,  0.14719999, ...,  0.01920529,
         0.04357562, -0.2117048 ],
       ...,
       [-0.43197954, -0.12469827,  0.4960655 , ..., -0.03080089,
        -0.08732568, -0.424841  ],
       [-0.41188836, -0.41201228, -0.07420321, ..., -0.04880745,
        -0.25071102, -0.47541225],
       [-0.39070803, -0.13806063,  0.43370932, ..., -0.00787478,
        -0.12150173, -0.66131675]], dtype=float32)

In [17]:
bert_matrix = _
bert_matrix.shape

(10000, 768)